In [58]:
#purpose finish foward and backward search algorithm and create a high efficient search algorithm
import pandas as pd
import numpy as np
import time
from sklearn import preprocessing
import sys
print ("this is my AI assginmant")


 this is my AI assginmant


In [59]:
def scale_normalized(data):
    max_num = max(data)
    min_num = min(data)
    for i,item in enumerate(data):
        data[i] = (data[i]-min_num)/max_num
    return data

In [100]:
dftrain = pd.read_csv("CS205_BIGtestdata__8.txt",delim_whitespace=True,header=None)
#dftrain = pd.read_csv("CS205_SMALLtestdata__70.txt",delim_whitespace=True,header=None)
#dftrain = pd.read_csv("CS205_SMALLtestdata__23.txt",delim_whitespace=True,header=None) #read the file and delimeter is whitespace
# header is none is different with header is None
total_features = dftrain.shape[1]#0 for row 1 for columns 
rows = dftrain.shape[0]
print (dftrain.shape)
labels = dftrain[dftrain.columns[0]]# keep the first column as labels
print ("--start normalizing, and it will take some time")
for i in range(1,total_features):#the orocess of normalized
    dftrain[dftrain.columns[i]]=scale_normalized(dftrain[dftrain.columns[i]])
print ("FINISH NORMALIZE")

(100, 51)
--start normalizing, and it will take some time
FINISH NORMALIZE


In [115]:
def leave_one_out_cross_validation(rows,k,test_data):
    res = 0 
    for i,item in enumerate(test_data):
        min_distance = 100
        distance_list = []
        index = -1
        for j,cmp_item in enumerate(test_data):
            distance =0.0
            if i==j:
                distance_list.append(100.0)
                continue
            for (x,y) in zip(item,cmp_item):
                distance += (x-y)**2
            distance_list.append(distance)
        min_distance = min(distance_list)
        label_index=-1
        for k, num in enumerate(distance_list):
            if num == min_distance:
                label_index = k
                break
        if labels[i] == labels[label_index]:#it means the prediction for this label is correct
            res +=1
    print ("The accuracy of data is :", res/100.00)
    return (res/100.00)

In [116]:
def call_test_data(temp_items):
    data = []
    sys.stdout.write("using features: ")
    sys.stdout.flush()
    print temp_items,
    
    #print ("using features: "temp_items,)
    for value in (dftrain.values):
        temp = []
        for item in temp_items:
            temp.append(value[item])
        data.append(temp)
    return list(data)

In [117]:
def forward_search(row,column):
    print ("start forward_search")
    items=[]
    lawyer_score = []
    start_time = time.time()
    for num in range (1,total_features):
        max_score = -1
        test_data = []
        new_element = -1
        for i in range (1,total_features):
            if i in items:
                continue
            else:
                temp_items=[]
                for element in items:# can not assgin temp_items = items, it will affect the items
                    temp_items.append(element)
                temp_items.append(i)
                test_data = call_test_data(temp_items)# based on the element number to get new test_data columns 
                cur_score = leave_one_out_cross_validation(row,num,test_data)# num is num of columns
                if cur_score > max_score:
                    new_element = i
                    max_score = cur_score
        print (max_score,new_element)
        lawyer_score.append(max_score)
        items.append(new_element)
    max_num = max(lawyer_score) # score the maximum scores
    index = -1 # score the best features 
    for i, num in enumerate(lawyer_score):
        if num == max_num:
            index = i+1 
            break
    print ("max score = " ,max_num)
    print ("feature = " ,items[0:index])
    print ("total time is %s seconds",time.time()-start_time)

In [118]:
def Backward_Elimination(row,column):
    print ("--start Backward_Elimination")
    start_time = time.time()
    print ("start recording time")
    items=[]
    remove_list = []
    for i in range(1,total_features):
        items.append(i)
    print (items)
    lawyer_score = []
    for num in range (1,total_features-1):
        max_score = -1 # store the best score in this lawyer
        min_score = 101
        test_data = []
        new_element = -1
        for i in range (1,total_features):
            if i not in items:
                continue
            else:
                temp_items=[]
                for element in items:# can not assgin temp_items = items, it will affect the items
                    temp_items.append(element)
                temp_items.remove(i)
                test_data = call_test_data(temp_items)# based on the element number to get new test_data columns 
                cur_score = leave_one_out_cross_validation(row,total_features-num-1,test_data)
                if cur_score > max_score:#delete the less
                    new_element = i
                    max_score = cur_score
        print (max_score,new_element)
        remove_list.append(new_element)
        lawyer_score.append(max_score)
        items.remove(new_element)
    max_num = max(lawyer_score) # score the maximum scores
    index = -1 # score the best features 
    for i, num in enumerate(lawyer_score):
        if num == max_num: # it can eliminate the feature as many as posiible 
            index = i+1 
            break
    print ("max score = " ,max_num)
    items=[]
    for i in range(1,total_features):
        items.append(i)
    for item in remove_list[0:index]:
        items.remove(item)
    print ("feature = " ,items)
    print ("total time is %s seconds",time.time()-start_time)

In [119]:
def original_leave_one_out_cross_validation(rows,k,test_data,max_score):
    # the idea is that if we find the score is lower than before, do not need to finish the calculation
    max_error = 100 - max_score*100
    error_rate = 0 
    res = 0 
    for i,item in enumerate(test_data):
        min_distance = 100
        distance_list = []
        index = -1
        for j,cmp_item in enumerate(test_data):
            distance =0.0
            if i==j:
                distance_list.append(100.0)
                continue
            if error_rate >= max_error:
                print ("The accuracy of data is :", 0) 
                return 0
            for (x,y) in zip(item,cmp_item):
                distance += (x-y)**2
            distance_list.append(distance)
        min_distance = min(distance_list)
        label_index=-1
        for k, num in enumerate(distance_list):
            if num == min_distance:
                label_index = k
                break
        if labels[i] == labels[label_index]:#it means the prediction for this label is correct
            res +=1
        else:
            error_rate +=1
    print ("The accuracy of data is :", res/100.00)
    return (res/100.00)

In [120]:
def original_search(row,total_features):
    print ("start recording time")
    start_time = time.time()
    print ("--THIS IS MY OWN SEARCH--")
    #basically, it is a forward search, but instead of leave_one_out_cross_validation this algorithm put concept of alpha 
    #beta inside the the leave one out ->  original_leave_one_out_cross_validation
    items=[]
    lawyer_score = []
    index_time = 0.0
    for num in range (1,total_features):
        max_score = -1
        test_data = []
        new_element = -1
        for i in range (1,total_features):
            if i in items:
                continue
            else:
                temp_items=[]
                for element in items:# can not assgin temp_items = items, it will affect the items
                    temp_items.append(element)
                temp_items.append(i)
                test_data = call_test_data(temp_items)# based on the element number to get new test_data columns 
                cur_score = original_leave_one_out_cross_validation(row,num,test_data,max_score)# num is num of columns
                if cur_score > max_score:
                    new_element = i
                    max_score = cur_score
        print (max_score,new_element)
        lawyer_score.append(max_score)
        items.append(new_element)
    max_num = max(lawyer_score)# score the maximum scores
    index = -1 # score the best features 
    for i, num in enumerate(lawyer_score):
        if num == max_num:
            index = i+1 
            break
    print ("max score = " ,max_num)
    print ("feature = " ,items[0:index])
    print ("total time is %s seconds",time.time()-start_time)

In [121]:
def main(): 
    print ("please input between 1-3 \n 1 for forward search \n 2 for backard_elemination \n 3 for original_search")
    choice = raw_input()
    if int(choice) == 1 :
        forward_search(rows,total_features)
    elif int(choice) == 2:
        Backward_Elimination(rows,total_features)
    elif int(choice) == 3:
        original_search(rows,total_features)
    else:
        print ("please input betweeen 1-3")
if __name__ == "__main__":
    main()

please input between 1-3 
 1 for forward search 
 2 for backard_elemination 
 3 for original_search
1
start forward_search
using features: [1] ('The accuracy of data is :', 0.77)
using features: [2] ('The accuracy of data is :', 0.72)
using features: [3] ('The accuracy of data is :', 0.73)
using features: [4] ('The accuracy of data is :', 0.78)
using features: [5] ('The accuracy of data is :', 0.72)
using features: [6] ('The accuracy of data is :', 0.66)
using features: [7] ('The accuracy of data is :', 0.7)
using features: [8] ('The accuracy of data is :', 0.75)
using features: [9] ('The accuracy of data is :', 0.75)
using features: [10] ('The accuracy of data is :', 0.78)
using features: [11] ('The accuracy of data is :', 0.74)
using features: [12] ('The accuracy of data is :', 0.71)
using features: [13] ('The accuracy of data is :', 0.76)
using features: [14] ('The accuracy of data is :', 0.86)
using features: [15] ('The accuracy of data is :', 0.77)
using features: [16] ('The accur

using features: [14, 6, 38] ('The accuracy of data is :', 0.79)
using features: [14, 6, 39] ('The accuracy of data is :', 0.81)
using features: [14, 6, 40] ('The accuracy of data is :', 0.81)
using features: [14, 6, 41] ('The accuracy of data is :', 0.81)
using features: [14, 6, 42] ('The accuracy of data is :', 0.78)
using features: [14, 6, 43] ('The accuracy of data is :', 0.85)
using features: [14, 6, 44] ('The accuracy of data is :', 0.83)
using features: [14, 6, 45] ('The accuracy of data is :', 0.8)
using features: [14, 6, 46] ('The accuracy of data is :', 0.8)
using features: [14, 6, 47] ('The accuracy of data is :', 0.83)
using features: [14, 6, 48] ('The accuracy of data is :', 0.75)
using features: [14, 6, 49] ('The accuracy of data is :', 0.82)
using features: [14, 6, 50] ('The accuracy of data is :', 0.85)
(0.9, 9)
using features: [14, 6, 9, 1] ('The accuracy of data is :', 0.83)
using features: [14, 6, 9, 2] ('The accuracy of data is :', 0.86)
using features: [14, 6, 9, 3]

using features: [14, 6, 9, 29, 18, 17] ('The accuracy of data is :', 0.81)
using features: [14, 6, 9, 29, 18, 19] ('The accuracy of data is :', 0.83)
using features: [14, 6, 9, 29, 18, 20] ('The accuracy of data is :', 0.86)
using features: [14, 6, 9, 29, 18, 21] ('The accuracy of data is :', 0.83)
using features: [14, 6, 9, 29, 18, 22] ('The accuracy of data is :', 0.81)
using features: [14, 6, 9, 29, 18, 23] ('The accuracy of data is :', 0.78)
using features: [14, 6, 9, 29, 18, 24] ('The accuracy of data is :', 0.86)
using features: [14, 6, 9, 29, 18, 25] ('The accuracy of data is :', 0.79)
using features: [14, 6, 9, 29, 18, 26] ('The accuracy of data is :', 0.78)
using features: [14, 6, 9, 29, 18, 27] ('The accuracy of data is :', 0.85)
using features: [14, 6, 9, 29, 18, 28] ('The accuracy of data is :', 0.83)
using features: [14, 6, 9, 29, 18, 30] ('The accuracy of data is :', 0.73)
using features: [14, 6, 9, 29, 18, 31] ('The accuracy of data is :', 0.8)
using features: [14, 6, 9,

using features: [14, 6, 9, 29, 18, 43, 2, 35] ('The accuracy of data is :', 0.75)
using features: [14, 6, 9, 29, 18, 43, 2, 36] ('The accuracy of data is :', 0.77)
using features: [14, 6, 9, 29, 18, 43, 2, 37] ('The accuracy of data is :', 0.76)
using features: [14, 6, 9, 29, 18, 43, 2, 38] ('The accuracy of data is :', 0.79)
using features: [14, 6, 9, 29, 18, 43, 2, 39] ('The accuracy of data is :', 0.77)
using features: [14, 6, 9, 29, 18, 43, 2, 40] ('The accuracy of data is :', 0.74)
using features: [14, 6, 9, 29, 18, 43, 2, 41] ('The accuracy of data is :', 0.8)
using features: [14, 6, 9, 29, 18, 43, 2, 42] ('The accuracy of data is :', 0.81)
using features: [14, 6, 9, 29, 18, 43, 2, 44] ('The accuracy of data is :', 0.74)
using features: [14, 6, 9, 29, 18, 43, 2, 45] ('The accuracy of data is :', 0.8)
using features: [14, 6, 9, 29, 18, 43, 2, 46] ('The accuracy of data is :', 0.77)
using features: [14, 6, 9, 29, 18, 43, 2, 47] ('The accuracy of data is :', 0.79)
using features: [1

using features: [14, 6, 9, 29, 18, 43, 2, 27, 47, 46] ('The accuracy of data is :', 0.82)
using features: [14, 6, 9, 29, 18, 43, 2, 27, 47, 48] ('The accuracy of data is :', 0.82)
using features: [14, 6, 9, 29, 18, 43, 2, 27, 47, 49] ('The accuracy of data is :', 0.8)
using features: [14, 6, 9, 29, 18, 43, 2, 27, 47, 50] ('The accuracy of data is :', 0.85)
(0.87, 15)
using features: [14, 6, 9, 29, 18, 43, 2, 27, 47, 15, 1] ('The accuracy of data is :', 0.84)
using features: [14, 6, 9, 29, 18, 43, 2, 27, 47, 15, 3] ('The accuracy of data is :', 0.81)
using features: [14, 6, 9, 29, 18, 43, 2, 27, 47, 15, 4] ('The accuracy of data is :', 0.8)
using features: [14, 6, 9, 29, 18, 43, 2, 27, 47, 15, 5] ('The accuracy of data is :', 0.88)
using features: [14, 6, 9, 29, 18, 43, 2, 27, 47, 15, 7] ('The accuracy of data is :', 0.85)
using features: [14, 6, 9, 29, 18, 43, 2, 27, 47, 15, 8] ('The accuracy of data is :', 0.79)
using features: [14, 6, 9, 29, 18, 43, 2, 27, 47, 15, 10] ('The accuracy 

...

using features: [14, 6, 9, 29, 18, 43, 2, 27, 47, 15, 16, 22, 3, 11, 46, 8, 25, 50, 28, 23, 30, 49, 17, 13, 7, 20, 33, 32, 21, 39, 10, 26] ('The accuracy of data is :', 0.82)
using features: [14, 6, 9, 29, 18, 43, 2, 27, 47, 15, 16, 22, 3, 11, 46, 8, 25, 50, 28, 23, 30, 49, 17, 13, 7, 20, 33, 32, 21, 39, 10, 31] ('The accuracy of data is :', 0.84)
using features: [14, 6, 9, 29, 18, 43, 2, 27, 47, 15, 16, 22, 3, 11, 46, 8, 25, 50, 28, 23, 30, 49, 17, 13, 7, 20, 33, 32, 21, 39, 10, 34] ('The accuracy of data is :', 0.8)
using features: [14, 6, 9, 29, 18, 43, 2, 27, 47, 15, 16, 22, 3, 11, 46, 8, 25, 50, 28, 23, 30, 49, 17, 13, 7, 20, 33, 32, 21, 39, 10, 35] ('The accuracy of data is :', 0.83)
using features: [14, 6, 9, 29, 18, 43, 2, 27, 47, 15, 16, 22, 3, 11, 46, 8, 25, 50, 28, 23, 30, 49, 17, 13, 7, 20, 33, 32, 21, 39, 10, 36] ('The accuracy of data is :', 0.88)
using features: [14, 6, 9, 29, 18, 43, 2, 27, 47, 15, 16, 22, 3, 11, 46, 8, 25, 50, 28, 23, 30, 49, 17, 13, 7, 20, 33, 32, 21,

using features: [14, 6, 9, 29, 18, 43, 2, 27, 47, 15, 16, 22, 3, 11, 46, 8, 25, 50, 28, 23, 30, 49, 17, 13, 7, 20, 33, 32, 21, 39, 10, 36, 19, 44] ('The accuracy of data is :', 0.82)
using features: [14, 6, 9, 29, 18, 43, 2, 27, 47, 15, 16, 22, 3, 11, 46, 8, 25, 50, 28, 23, 30, 49, 17, 13, 7, 20, 33, 32, 21, 39, 10, 36, 19, 45] ('The accuracy of data is :', 0.82)
using features: [14, 6, 9, 29, 18, 43, 2, 27, 47, 15, 16, 22, 3, 11, 46, 8, 25, 50, 28, 23, 30, 49, 17, 13, 7, 20, 33, 32, 21, 39, 10, 36, 19, 48] ('The accuracy of data is :', 0.86)
(0.86, 1)
using features: [14, 6, 9, 29, 18, 43, 2, 27, 47, 15, 16, 22, 3, 11, 46, 8, 25, 50, 28, 23, 30, 49, 17, 13, 7, 20, 33, 32, 21, 39, 10, 36, 19, 1, 4] ('The accuracy of data is :', 0.81)
using features: [14, 6, 9, 29, 18, 43, 2, 27, 47, 15, 16, 22, 3, 11, 46, 8, 25, 50, 28, 23, 30, 49, 17, 13, 7, 20, 33, 32, 21, 39, 10, 36, 19, 1, 5] ('The accuracy of data is :', 0.83)
using features: [14, 6, 9, 29, 18, 43, 2, 27, 47, 15, 16, 22, 3, 11, 46

using features: [14, 6, 9, 29, 18, 43, 2, 27, 47, 15, 16, 22, 3, 11, 46, 8, 25, 50, 28, 23, 30, 49, 17, 13, 7, 20, 33, 32, 21, 39, 10, 36, 19, 1, 35, 24, 41] ('The accuracy of data is :', 0.81)
using features: [14, 6, 9, 29, 18, 43, 2, 27, 47, 15, 16, 22, 3, 11, 46, 8, 25, 50, 28, 23, 30, 49, 17, 13, 7, 20, 33, 32, 21, 39, 10, 36, 19, 1, 35, 24, 42] ('The accuracy of data is :', 0.78)
using features: [14, 6, 9, 29, 18, 43, 2, 27, 47, 15, 16, 22, 3, 11, 46, 8, 25, 50, 28, 23, 30, 49, 17, 13, 7, 20, 33, 32, 21, 39, 10, 36, 19, 1, 35, 24, 44] ('The accuracy of data is :', 0.81)
using features: [14, 6, 9, 29, 18, 43, 2, 27, 47, 15, 16, 22, 3, 11, 46, 8, 25, 50, 28, 23, 30, 49, 17, 13, 7, 20, 33, 32, 21, 39, 10, 36, 19, 1, 35, 24, 45] ('The accuracy of data is :', 0.77)
using features: [14, 6, 9, 29, 18, 43, 2, 27, 47, 15, 16, 22, 3, 11, 46, 8, 25, 50, 28, 23, 30, 49, 17, 13, 7, 20, 33, 32, 21, 39, 10, 36, 19, 1, 35, 24, 48] ('The accuracy of data is :', 0.78)
(0.84, 5)
using features: [14,

using features: [14, 6, 9, 29, 18, 43, 2, 27, 47, 15, 16, 22, 3, 11, 46, 8, 25, 50, 28, 23, 30, 49, 17, 13, 7, 20, 33, 32, 21, 39, 10, 36, 19, 1, 35, 24, 5, 34, 38, 37, 4] ('The accuracy of data is :', 0.8)
using features: [14, 6, 9, 29, 18, 43, 2, 27, 47, 15, 16, 22, 3, 11, 46, 8, 25, 50, 28, 23, 30, 49, 17, 13, 7, 20, 33, 32, 21, 39, 10, 36, 19, 1, 35, 24, 5, 34, 38, 37, 12] ('The accuracy of data is :', 0.85)
using features: [14, 6, 9, 29, 18, 43, 2, 27, 47, 15, 16, 22, 3, 11, 46, 8, 25, 50, 28, 23, 30, 49, 17, 13, 7, 20, 33, 32, 21, 39, 10, 36, 19, 1, 35, 24, 5, 34, 38, 37, 26] ('The accuracy of data is :', 0.85)
using features: [14, 6, 9, 29, 18, 43, 2, 27, 47, 15, 16, 22, 3, 11, 46, 8, 25, 50, 28, 23, 30, 49, 17, 13, 7, 20, 33, 32, 21, 39, 10, 36, 19, 1, 35, 24, 5, 34, 38, 37, 31] ('The accuracy of data is :', 0.83)
using features: [14, 6, 9, 29, 18, 43, 2, 27, 47, 15, 16, 22, 3, 11, 46, 8, 25, 50, 28, 23, 30, 49, 17, 13, 7, 20, 33, 32, 21, 39, 10, 36, 19, 1, 35, 24, 5, 34, 38, 3

using features: [14, 6, 9, 29, 18, 43, 2, 27, 47, 15, 16, 22, 3, 11, 46, 8, 25, 50, 28, 23, 30, 49, 17, 13, 7, 20, 33, 32, 21, 39, 10, 36, 19, 1, 35, 24, 5, 34, 38, 37, 12, 31, 26, 41, 45] ('The accuracy of data is :', 0.78)
using features: [14, 6, 9, 29, 18, 43, 2, 27, 47, 15, 16, 22, 3, 11, 46, 8, 25, 50, 28, 23, 30, 49, 17, 13, 7, 20, 33, 32, 21, 39, 10, 36, 19, 1, 35, 24, 5, 34, 38, 37, 12, 31, 26, 41, 48] ('The accuracy of data is :', 0.8)
(0.84, 42)
using features: [14, 6, 9, 29, 18, 43, 2, 27, 47, 15, 16, 22, 3, 11, 46, 8, 25, 50, 28, 23, 30, 49, 17, 13, 7, 20, 33, 32, 21, 39, 10, 36, 19, 1, 35, 24, 5, 34, 38, 37, 12, 31, 26, 41, 42, 4] ('The accuracy of data is :', 0.8)
using features: [14, 6, 9, 29, 18, 43, 2, 27, 47, 15, 16, 22, 3, 11, 46, 8, 25, 50, 28, 23, 30, 49, 17, 13, 7, 20, 33, 32, 21, 39, 10, 36, 19, 1, 35, 24, 5, 34, 38, 37, 12, 31, 26, 41, 42, 40] ('The accuracy of data is :', 0.81)
using features: [14, 6, 9, 29, 18, 43, 2, 27, 47, 15, 16, 22, 3, 11, 46, 8, 25, 50, 